In [8]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine


In [9]:
def cargaMySql (nombre, dataset):
    a = nombre
    nombre = dataset
    
    #Creacion de variables
    host_name,db_name, u_name, u_pass, port_num  = "localhost","proyecto_final", "root", "cr4zyfr0g", "3306"
    
    #Conexion a la db
    mydb = mysql.connector.connect(
    host=host_name,
    user=u_name,
    password=u_pass,
    database=db_name )
    #Creacion del motor de base de datos
    engine = create_engine("mysql+mysqlconnector://" + u_name + ":" + u_pass + "@" + host_name + ":" + port_num + "/" + db_name, echo=False)
    
    #Importacion del dataframe a nuestra base de datos en SQL
    nombre.to_sql(name=a, con=engine, if_exists="append", index=False)
    return print('Carga exitosa')

In [10]:
def calidaDato (data):
    df = data
    calidad = []
    mascara = df.isna().sum()
    for i in range(0,len(mascara)):
        p = (df.shape[0]- mascara[i]) / df.shape[0]
        calidad.append([mascara.index[i], round((p*100),2)])
   
    calidad = pd.DataFrame(calidad)
    calidad.rename(columns={0:'Columna', 1: 'Calida %'}, inplace=True)
    
    return calidad

In [ ]:
def generarId (df, colmna):
    df = df.sort_values(by=colmna)
    n = 0
    nombre = "Id" + colmna
    df[nombre] = n
    for i in df[colmna].unique():
        n += 1
        df.loc[df[colmna] == i, nombre] = n
    return df

In [ ]:
def limpieza (df):
    translator = Translator()
    for i in df.columns:
        df.rename(columns={i:translator.translate(i, dest="es").text.title()}, inplace=True)
    df.drop(columns="Sin Nombre: 0", inplace=True)

    otro = pd.read_csv("./owid-co2-data.csv", delimiter=",", encoding="utf-8")
    otro = otro[["year","country","population","gdp"]]
    df = pd.merge(df, otro, left_on=["Año","País"], right_on=["year","country"], how="left")
    df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False), "Población"] = df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False)]["population"]
    df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False), "Pib"] = df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False)]["gdp"]
    df.drop(columns=["country", "year", "population","gdp"], inplace=True)

    df.dropna(subset=["Emisión De Co2"], inplace=True)

    for i in df.columns:
        if i == "Tipo_Energia":
            continue
        elif "Energ" in i:
            nom = i + " " + "TWh"
            df[nom] = df[i] / 3.412e+12

    return df

In [ ]:
def prueba(path):
    with os.scandir(path) as ficheros:
        for i in ficheros:
            directorio, extencion = os.path.splitext(i)
            if extencion == "":
                prueba(directorio)
                continue
            else:
                if extencion == ".txt" or extencion == ".csv":
                    ruta = directorio + extencion
                    arch = os.path.basename(ruta)
                    df = DataFrame2(ruta)
                    baseDeDatos(df,arch[:-4])

In [ ]:
#funcion para subir un archivo a una carpeta en especifico
def subircsv(nomcarpeta, nomarchivo, extencion, df):
    df = df
    filepath = Path(nomcarpeta + '/' + nomarchivo +'.' + extencion)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath)  

In [ ]:
def renombracol(data):
    data = data
    translator = Translator()
    for i in data.columns:
        b = i.replace('_',' ')
        data.rename(columns={i:b}, inplace=True)
        data.rename(columns={'owner':'owne'}, inplace=True)
        
    for i in data.columns:
        data.rename(columns={i:translator.translate(i, dest="es").text.title()}, inplace=True)
        
    return data

In [ ]:
def subircsv(nomcarpeta, nomarchivo, extencion, df):
    df = df
    filepath = Path(nomcarpeta + '/' + nomarchivo +'.' + extencion)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath)

In [ ]:
import hashlib
def hash(df,colum):
    nom = "Id" + colum
    if df[colum].dtype == 'int64':
        df[colum] = df[colum].astype(str)
        df[nom] = df[colum].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        df[colum] = df[colum].astype(int)
    else:
        df[nom] = df[colum].apply(lambda x: hashlib.md5(x.encode()).hexdigest())